# Выпускная квалификационная работа
### на тему «Применение машинного обучения для категоризации тестовых данных»
Выполнил студент группы ЗБ-ПИ20(2) Миловидов В.И.

In [ ]:
pip install openpyxl

In [ ]:
pip install tabulate

In [ ]:
pip install wordcloud

In [ ]:
pip install -U scikit-learn

In [ ]:
pip install optree

In [ ]:
pip install --upgrade optree

In [ ]:
pip install --upgrade umap-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from openpyxl import load_workbook
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import timeit
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
import sklearn_crfsuite
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import learning_curve

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
%%time
# Шаг 1: Чтение данных из xlsx файла
workbook = load_workbook(filename='dat_min_500_61k.xlsx')
sheet = workbook.active
data = []
for row in sheet.iter_rows(values_only=True):
    data.append(row)

df = pd.DataFrame(data, columns=['text', 'product'])

## Анализ данных

In [ ]:
# выводим данные
df

In [ ]:
%%time
# Вывод статистической сводки датафрейма (Выявление аномальных значений)
table = tabulate(df.describe(), headers='keys', tablefmt='psql')
print(table)

In [ ]:
%%time
# Вывод шкалы измерения признака "Продукт"
print("Уникальные значения 'product':", df['product'].nunique())
print(df['product'].value_counts())


Из информации выше можно сделать следующие выводы:
- В колонке "product" имеется 28 уникальных значения.
- Самым частым значением в колонке "product" является "Оплата картой (Прямые мерчанты)" с частотой 15430.
- Значение "СБП" встречается 8066 раз.
- Также отмечается, что значение "SberPay" с наименьшей частотой составляет 158.


In [ ]:
%%time
# Вывод обобщенной информации о датафрейме
info_table = tabulate(df.info(), headers='keys', tablefmt='psql')
print(info_table)

In [ ]:
%%time
# Проверка на пропущенные/пустые значения
print(df.isnull().sum())

## Описательный анализ

In [ ]:
%%time
#Визуализация распределения признака 'product'
plt.figure(figsize=(10, 8))

# Построение диаграммы
df['product'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
%%time
#Выявление коррелирующих признаков и признаков, не несущих информации для данной задачи
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vec = TfidfVectorizer()
X = vec.fit_transform(df['text'])
y = df['product']

model = LogisticRegression()
model.fit(X, y)

# Посмотрим на важность каждого слова
importances = pd.DataFrame(data=model.coef_[0], index=vec.get_feature_names_out(), columns=['importance'])
importances.sort_values('importance', ascending=False, inplace=True)
print(importances)
importances.to_excel('importances.xlsx')
print("Файл 'importances.xlsx' успешно сохранен")

Из предоставленной информации, которая содержит значения и их важность (importance), можно сделать следующие выводы:

1. Высокая важность: Некоторые значения имеют высокую важность, такие как "проверяется" (importance: 3.427926), "карты" (importance: 3.141554), "отклонения" (importance: 2.693837), "дс" (importance: 2.650492) и "кода" (importance: 2.618930). Высокая важность этих значений может указывать на их значимость или влияние на анализ или обработку данных.
2. Низкая важность: Некоторые значения имеют низкую важность, такие как "сумму" (importance: -1.051133), "иэ" (importance: -1.142789), "на" (importance: -1.164227), "сбп" (importance: -1.178481) и "возврат" (importance: -2.073418). Низкая важность может указывать на то, что эти значения имеют меньшее влияние на анализ или обработку данных.
3. Влияние на принятие решений: при анализе полного списка значения с высокими и низкими показателями важности имели большое влияние на принятие решени. Значения с выскоим importance - встали в основу "ключевых слов" при преварительной обработке текста. Значения с низким importance - заложили основу стоп-слов в иденичном процессе


In [ ]:
%%time
#Word Cloud (Облако слов) - визуализация
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Объединяем все текстовые сообщения в одну строку
text = ' '.join(df['text'])

# Создаем Word Cloud и отображаем его
wordcloud = WordCloud(width=800, height=400).generate(text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
#Heatmap (Тепловая карта)
import seaborn as sns
import matplotlib.pyplot as plt


# Создаем тепловую карту
heatmap_data = df.groupby(['text', 'product']).size().unstack(fill_value=0)
sns.heatmap(heatmap_data, cmap='viridis')
plt.show()

Вывод: при текущий значениях - метод не релевантен для анализа

## Предообработка данных
В этом шаге проведем предварительную обработку текста, чтобы подготовить его к дальнейшему применению. Мы будем использовать некоторые основные методы обработки текста, включая удаление стоп-слов и лемматизацию слов.

1. Сначала мы инициализируем список стоп-слов на русском языке с помощью stop_words = list(stopwords.words('russian'))
2. Затем дополняем список стоп-слов некоторыми дополнительными фразами и словами, которые хотим исключить из текста с помощью stop_words.extend([...])
3. Затем вводим функцию preprocess_text(text), которая принимает текст в качестве входного аргумента. Внутри этой функции выполняем следующие действия:
    - Преобразуем текст в нижний регистр с помощью text.lower().
    - Разбиваем текст на отдельные слова (токены) с использованием word_tokenize().
    - Производим лемматизацию каждого токена с помощью объекта lemmatizer.lemmatize(token).
    - Оставляем только буквенно-цифровые токены с помощью if token.isalnum().
    - Исключаем стоп-слова из списка токенов с помощью if token not in stop_words.
    - Добавляем ключевые слова к списку токенов, чтобы учитывать их при анализе.
    - Формируем окончательную строку путем объединения токенов с помощью ' '.join(tokens).
4. Применяем данную функцию к столбцу "text" в DataFrame df и сохраняем результат в столбце "Processed_Text"


In [ ]:
%%time
# Шаг 2: Предварительная обработка текста
stop_words = list(stopwords.words('russian'))

stop_words.extend([
 'С уважением',
 'Группа МПС',
 'МПС',
 'Отдел сопровождения',
 'ПЦ',
 'Контакты',
 'Вернемся с ответом в понедельник до по МСК',
 'Вернемся с ответом',
 'МСК',
 'до',
 'по',
 'Мы зарегистрировали ваше обращение и возьмем его в работу в ближайшее время',
 'Номер заявки',
 'Ответы на популярные вопросы можно найти в Тинькофф Помощи',
 'Команда поддержки Тинькофф Касса',
 'Январь',
 'Февраль',
 'Март',
 'Апрель',
 'Май',
 'Июнь',
 'Июль',
 'Август',
 'Сентябрь',
 'Октябрь',
 'Ноябрь',
 'Декабрь',
 'ФИО',
 'Используй кейс',
 'Сразу добавь в копию почту почты из столбца справа если указано',
 'Комментарий',
 'Обращение создано через формы на',
 'Просьба',
 'Менеджер по развитию партнеров',
 'Отдела продаж',
 'Департамента платежных систем',
 'Наталия Алексуткина',
 'Моб',
 'Доп',
 'Это электронное письмо строго конфиденциально включая все вложения',
 'Ссылка на чат с клиентом',
 'Отсутствует',
 'Власов Александр',
 'Технический специалист',
 'Мы в и',
 'Информация содержащаяся в данном письме и приложенных к нему материалах составляет коммерческую тайну и является конфиденциальной в связи с чем воспроизведение копирование распространение или использование информации любым иным образом может осуществляться исключительно с согласия уполномоченного представителя',
 'Если данное письмо было получено вами ошибочно просим незамедлительно удалить письмо',
 'пожалуйста',
 'нажимайте на ответить всем не видно ваших ответов',
 'Менеджер по развитию бизнеса',
 'Менеджер по развитию',
 'Команда сопровождения платежей и переводов',
 'спасибо',
 'Заранее',
 'посмотрите',
 'запрос',
 'Уточните',
 'Отдел сверок',
 'Ссылка на чат с клиентом',
 'Возобновление задачи',
 'Можем',
 'Мы',
 'Руководитель проектов',
 'Департамент платежных систем',
 'Ростелеком',
 'Вопрос в работе',
 'Вернемся с ответом до',
 'Прошу',
 'В чем может быть проблема',
 'Поддержка продуктов для бизнеса',
 'Бэк офис',
 'скриншот',
 'прикрепляю',
 'Сообщите',
 'номер тикета',
 'Специалист по работе с ключевыми клиентами',
 'необходимо',
 'подскажите',
 'умеем',
 'помогите',
 'поступило обращение',
 'поддержка',
 'Полиса',
 'Дата и время события',
 'Подробное описание',
 'Пользуйся кейсами со страницы продукта',
 'Подробности',
 'Проблема',
 'Отправлено из Почты',
 'Исходное сообщение',
 'поставьте нам оценку по ссылке',
 'уточнить',
 'Группа контроля эквайринга',
 'Данное сообщение отправлено из',
 'В случае отсутствия доп вопросов к',
 'не отвечать на него',
 'а так же удалить рассылку из дальнейшей переписки',
 'можете',
 'подсказать',
 'с вашей стороны',
 'Старший специалист',
 'вопрос',
 'Инженер группы поддержки платежей',
 'Отдел сопровождения платежей и переводов',
 'вы',
 'оценить мою работу здесь',
 'оставить чаевые здесь',
 'Менеджер по работе с партнерами',
 'прощение',
 'прощения',
 'пишу',
 'Сектор поддержки бизнеса',
 'Отдел претензий',
 'Управление',
 'Департамент',
 'Департамент клиентского обслуживания',
 'для звонков из России',
 'для звонков из-за границы',
 'Факс',
 'из-за',
 'из за',
 'чат',
 'Я ваш клиент',
 'Заранее благодарю',
 'Приносим извинения за доставленные неудобства',
 'извинения',
 'Диалог',
 'Персональный менеджер',
 'Направление корпоративного бизнеса',
 'Моб',
 'Бизнес аналитик',
 'аналитик',
 'добрый день',
 'здравствуйте',
 'добрый',
 'привет',
 'коллеги',
 'тел',
 'доб',
 'доброе утро',
 'доброе',
 'добрый вечер',
 'колеги'])



lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    start_time = timeit.default_timer()
    if text is not None:
        tokens = word_tokenize(text.lower())
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]
        tokens = [token for token in tokens if token not in stop_words]

    # Добавление ключевых слов
    keywords = [
 'qr',
 'sbp',
 'notifications',
 'notification',
 'cloudkassir',
 'rrn',
 'api',
 'iss_serno',
 'sdk',
 'cms',
 'tinkoff-pay',
 'tinkoffpay',
 'mirpay',
 'tinkoff pay',
 'returns amount',
 'msm',
 'acq',
 'acqapi',
 'compliance',
 'ovd',
 'tilda',
 'bitrix',
 'orangedata',
 'orange data',
 'ofd ferma',
 'ferma',
 'ecomkassa',
 'advantshop',
 'amiro',
 'amocrm',
 'cs-cart',
 'diafan',
 'drupal commerce',
 'drupal ubercart',
 'ecshop',
 'ecwid',
 'getcourse',
 'joomla hikashop',
 'hostcms',
 'image cms',
 'insales',
 'joomla',
 'joomshopping',
 'virtuemart',
 'magento',
 'maxystore',
 'modx shopkeeper',
 'mogutacms',
 'netcat',
 'opencart',
 'oscommerce',
 'phpshop.cms',
 'prestashop',
 'quickbuy',
 'simpla',
 'taplink',
 'umi.cms',
 'vamshop',
 'vigbo',
 'webasyst',
 'wfolio',
 'wordpress ecommerce',
 'wordpress shop',
 'wordpress',
 'woocommerce',
 'seller',
 'chargeback',
 'mapi',
 'arn',
 'ssl',
 'mcc',
 'bnpl',
 'outward',
 'outwards',
 'atol',
 'webhook',
 'wl',
 'white list',
 'whitelist',
 'dolyame']

    tokens += keywords

    return ' '.join(tokens)

df['Processed_Text'] = df['text'].apply(preprocess_text)

## Применение машинного обучения

In [ ]:
%%time
# Шаг 3: Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(df['Processed_Text'], df['product'], test_size=0.2, random_state=42)


Алгоритмы для классификации текста, которые будут проанализированы далее:
- Логистическая регрессия
- Метод k ближайших соседей (k-Nearest Neighbors, k-NN)
- Метод наивного Байеса (Naive Bayes)
- Методы опорных векторов (Support Vector Machines, SVM)
- Решающие деревья и случайный лес
- Градиентный бустинг (Gradient Boosting)
- Нейронные сети
- Метод максимальной энтропии (Maximum Entropy)
- Методы гибридной классификации

### Логистическая регрессия (LogisticRegression)

In [ ]:
%%time
# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Обучение модели логистической регрессии
classifier = LogisticRegression()
classifier.fit(X_train_vect, y_train)

# Шаг 6: Оценка модели
y_pred_lr = classifier.predict(X_test_vect)
print("Отчет модели логистической регресии:")
print(classification_report(y_test, y_pred_lr))

Можно сделать следующие выводы:

1. Производительность модели по классам: Модель показывает высокие значения F1-меры для некоторых классов, таких как "Онлайн-касса", "Реестры, акты и выгрузки", "Рассрочка на платежной форме", "Оплата картой (Прямые мерчанты)". Это свидетельствует о хорошей способности модели распознавать и классифицировать тексты, относящиеся к этим классам.
2. Развитие точности и полноты: Значения точности (precision) и полноты (recall) для различных классов могут отличаться. Например, для класса "Оплата картой (Прямые мерчанты)" значение точности составляет 0.53, в то время как значение полноты составляет 0.83.
    - Precision (Точность): Precision представляет собой долю правильно классифицированных положительных образцов относительно всех образцов, которые модель классифицировала как положительные. Судя по результатам, модель продемонстрировала высокую точность (precision) для некоторых классов, таких как "Tinkoff Pay Wallet" (0.98), "Tinkoff ID" (0.98), "Рассрочка на платежной форме" (0.99). Это говорит о том, что модель имеет небольшую вероятность ложной классификации для этих классов.
    - Recall (Полнота): Recall отражает способность модели правильно идентифицировать положительные образцы относительно всех фактически положительных образцов в наборе данных. Некоторые классы, такие как "Онлайн-касса" (0.82), "Оплата картой (Прямые мерчанты)" (0.83), "Рассрочка на платежной форме" (0.91), показывают высокую полноту. Это означает, что модель не пропускает много положительных образцов, связанных с этими классами.
    - F1-score: F1-score является средневзвешенной метрикой, объединяющей точность и полноту, которая учитывает баланс между этими двумя метриками. В целом, средневзвешенное значение F1-меры составляет около 0.68, что говорит о сбалансированных результатов модели по всем классам. Высокие значения F1-score наблюдаются для классов с высокой точностью и полнотой. Некоторые классы с высоким F1-score включают "Рассрочка на платежной форме" (0.95), "Онлайн-касса" (0.75), "Реестры, акты и выгрузки" (0.80). Однако некоторые классы, такие как "3DS", "Маркетплейс (переводы)", "Вне компетенции поддержки ИЭ", имеют очень низкие значения F1-score (меньше 0.10). Это может указывать на трудности в классификации этих конкретных классов.
4. Улучшение производительности: Чтобы улучшить производительность модели, можно рассмотреть следующие подходы: Использование более сложной модели или композиции моделей, которые могут лучше учитывать особенности текстовых данных.


Вывод: Результаты статистики свидетельствуют о разной производительности модели для разных классов. Некоторые классы демонстрируют высокую точность, полноту и F1-score, что говорит о хорошей способности модели правильно классифицировать эти классы. Однако, некоторые классы имеют низкие значения этих метрик, что может указывать на сложности и недостаток данных для этих классов. Для улучшения производительности модели с учетом особенностей и требований нашей задачи, рассмортим следующий алгоритм


### Метод k ближайших соседей (k-Nearest Neighbors, k-NN)

In [ ]:
%%time
# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Обучение модели k-Nearest Neighbors
knn = KNeighborsClassifier()
knn.fit(X_train_vect, y_train)

# Шаг 6: Оценка модели
y_pred_knn = knn.predict(X_test_vect)
print("Отчет модели k ближайших соседей:")
print(classification_report(y_test, y_pred_knn))

Можно сделать следующие выводы:

1. Точность (precision): Модели имеют разную точность для разных классов. Некоторые классы имеют высокую точность (например, "Маркетплейс (переводы)", "Модули CMS и виджеты", "Онлайн-касса"), в то время как другие классы имеют низкую точность (например, "3DS", "СБП E2C (Выплаты)", "Вне компетенции поддержки ИЭ"). Средняя точность по всем классам составляет около 0.54, что означает, что в среднем модели правильно классифицируют около 54% примеров.
2. Полнота (recall): Подобно точности, модели имеют разную полноту для разных классов. Некоторые классы имеют высокую полноту (например, "Тинькофф Pay Wallet", "Реестры, акты и выгрузки", "СБП"), в то время как другие классы имеют низкую полноту (например, "3DS", "Маркетплейс (переводы)", "Вне компетенции поддержки ИЭ"). Средняя полнота по всем классам составляет около 0.44, что означает, что в среднем модели правильно распознают около 44% примеров.
3. F1-мера (f1-score): F1-мера является сбалансированной метрикой, учитывающей и точность, и полноту. Поэтому она может дать представление о совокупной производительности модели. Средняя F1-мера по всем классам составляет около 0.47, что означает, что в среднем модели достигают F1-меры около 47%.

Вывод: сравнивая два метода Логистическая регрессия, где средний F1-score = 69% и Метод k ближайших соседей (k-Nearest Neighbors, k-NN), где средний F1-score = 59% - можем сделать вывод, что Логическая регрессия определяет классы с более точным показателем (будем сравнивать его с последющему алгоритмами)


### Метод наивного Байеса (Naive Bayes)

In [ ]:
%%time
# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Обучение модели Метода наивного Байеса
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_vect, y_train)

# Шаг 6: Оценка модели
y_pred_nb = naive_bayes.predict(X_test_vect)
print("Отчет модели наивного Байеса:")
print(classification_report(y_test, y_pred_nb))


Можно сделать следующие выводы:

1. Точность (precision): Большинство классов имеют низкую точность, равную нулю или близкую к нулю. Ограниченное количество классов имеют высокую точность, близкую к 1.0. Взвешенное среднее значение точности составляет около 0.60, что указывает на относительно низкую точность модели в целом.
2. Полнота (recall): Идентичная ситуация, как и с точностью. Взвешенное среднее значение полноты составляет около 0.41, что указывает на относительно низкую полноту модели в целом.
3. F1-мера (f1-score): Среднее взвешенное значение F1-меры составляет около 0.34, что указывает на относительно низкую сбалансированную производительность модели.

Таким образом, анализируя предоставленную статистику, можно сделать вывод, что модели имеют проблемы с предсказанием многих классов, показывая низкую точность, полноту и F1-меру. Некоторые классы могут быть обнаружены с высокой точностью, но здесь таких классов явно мало.


### Методы опорных векторов (Support Vector Machines, SVM)

In [ ]:
%%time
# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Обучение модели Методов опорных векторов (SVM)
svm = SVC()
svm.fit(X_train_vect, y_train)

# Шаг 6: Оценка модели
y_pred_svm = svm.predict(X_test_vect)
print("Отчет модели опорных векторов:")
print(classification_report(y_test, y_pred_svm))


Можно сделать следующие выводы:

1. Точность (precision): Некоторые классы имеют высокую точность (например, "3DS", "BNPL (Долями)", "E-inv", "SberPay", "Tinkoff ID", "Tinkoff Pay Wallet", "Безопасная сделка", "Выплаты (A2C)", "Кастомизация ПФ", "Онлайн-касса", "СБП", "СБП E2C (Выплаты)"), превышающую 0.8. Некоторые классы имеют низкую точность (например, "Вне компетенции поддержки ИЭ", "Маркетплейс (переводы)"), близкую к нулю или ниже 0.5. Средняя точность по всем классам составляет около 0.74, что указывает на хорошую общую точность модели.
2. Полнота (recall): Некоторые классы имеют высокую полноту (например, "Origination", "Tinkoff Pay Wallet", "Модули CMS и виджеты", "Онлайн-касса", "СБП"), превышающую 0.7. Некоторые классы имеют низкую полноту (например, "Вне компетенции поддержки ИЭ", "Маркетплейс (переводы)"), близкую к нулю или ниже 0.3. Средняя полнота по всем классам составляет около 0.70, что указывает на среднюю общую полноту модели.
3. F1-мера (f1-score): Многие классы имеют высокие значения F1-меры, которые близки к 1.0 (например, "Origination", "Tinkoff Pay Wallet", "Модули CMS и виджеты", "Онлайн-касса", "СБП"). Некоторые классы имеют низкие значения F1-меры, близкие к нулю или ниже 0.3 (например, "Вне компетенции поддержки ИЭ", "Маркетплейс (переводы)"). Средняя F1-мера по всем классам составляет около 0.70, что указывает на хорошую сбалансированную производительность модели.

Таким образом, метод опорных векторов (Support Vector Machines, SVM) превзошел преыдущие алгоритмы. Сравним в наилучшим показатлем по Логистической регрессии, где f1-score = 69%, против текущий 70%. План-base, что был поставлен в рамках требований к проекту, успешно выполнен. Следующая итерация - добиться максимального улучшения предикшина путем перебора альтернативных методов


### Решающих деревьев (DecisionTreeClassifier)


In [ ]:
%%time

# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Обучение модели Решающих деревьев
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train_vect, y_train)

# Шаг 6: Оценка модели Решающих деревьев
y_pred_decision_tree = decision_tree.predict(X_test_vect)
print("Отчет модели Решающих деревьев:")
print(classification_report(y_test, y_pred_decision_tree))

Можно сделать следующие выводы:
1. Процент правильно классифицированных образцов (accuracy) составляет 59%, что означает, что модель способна правильно предсказывать класс в примерно 59% случаев.
2. Метрика precision (точность), измеряющая долю верно предсказанных положительных классов среди всех положительных предсказаний, имеет среднее значение 49%
3. Метрика recall (полнота), измеряющая способность модели верно классифицировать все действительные положительные классы, имеет среднее значение 45%
4. Метрика f1-score (F1-мера), которая является гармоническим средним между точностью и полнотой, имеет среднее значение 59%

Вывод: модель имеет средние результаты в предсказании классов, с переменными результатами в зависимости от конкретного класса.


### Случайный лес (RandomForestClassifier)

In [ ]:
# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Обучение модели Случайного леса
random_forest = RandomForestClassifier()
random_forest.fit(X_train_vect, y_train)

# Шаг 6: Оценка модели Случайного леса
y_pred_random_forest = random_forest.predict(X_test_vect)
print("Отчет модели Случайного леса:")
print(classification_report(y_test, y_pred_random_forest))


Можно сделать следующие выводы:

1. Recall (полнота) модели Случайного леса различается в зависимости от класса. Некоторые классы имеют низкое значение полноты, что означает, что модель определяет только малую часть реально положительных случаев для этих классов.
2. F1-мера, объединяющая точность и полноту, также показывает смешанные результаты для различных классов. Некоторые классы имеют низкую F1-меру, что указывает на недостаточно хорошее баланс между точностью и полнотой. Общая точность (accuracy) модели составляет 59%, что означает, что около 59% образцов были правильно классифицированы моделью.

Вывод: модель Случайного леса имеет смешанные результаты производительности для разных классов.


### Градиентный бустинг (Gradient Boosting)

In [ ]:
%%time

# Шаг 3: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 4: Обучение модели градиентного бустинга
gradient_boosting = GradientBoostingClassifier()
gradient_boosting.fit(X_train_vect, y_train)

# Шаг 5: Оценка модели градиентного бустинга
y_pred_gradient_boosting = gradient_boosting.predict(X_test_vect)
print("Отчет модели градиентного бустинга:")
print(classification_report(y_test, y_pred_gradient_boosting))


Вывод:
- Было сделано 12387 предсказаний с использованием модели градиентного бустинга.
- Общая точность модели составляет 0.64, что означает, что около 64% предсказаний были классифицированы правильно.
- Для большинства категорий наибольшие значения точности, полноты и F1-меры наблюдаются для категорий, таких как "Tinkoff Pay Wallet" (0.93), "Рассрочка на платежной форме" (0.93) и "Онлайн-касса" (0.79), что указывает на хорошую эффективность модели при предсказании этих категорий.
- С другой стороны, некоторые категории имеют низкие значения точности, полноты и F1-меры. Например, категории "3DS", "СБП E2C (Выплаты)" и "Маркетплейс (переводы)" имеют очень низкую точность и полноту, что указывает на сложности с классификацией этих категорий для модели градиентного бустинга.

В целом, средние значения точности, полноты и F1-меры (macro avg) составляют 0.47, что означает, что модель имеет средние результаты в общем смысле.
Средневзвешенная точность, полнота и F1-мера (weighted avg) также составляют 0.64, что отражает средние значения метрик с учетом дисбаланса классов.
В общем и целом, модель градиентного бустинга показывает средние результаты в предсказании различных категорий. Из отчета видно, что некоторые категории успешно классифицируются моделью, в то время как у других категорий есть проблемы с точностью и полнотой классификации.


### Нейронные сети


In [ ]:
%%time
# Шаг 3: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 4: Обучение модели нейронных сетей
mlp = MLPClassifier()
mlp.fit(X_train_vect, y_train)

# Шаг 5: Оценка модели нейронных сетей
y_pred_mlp = mlp.predict(X_test_vect)
print("Отчет модели нейронных сетей:")
print(classification_report(y_test, y_pred_mlp))


Вывод:
- Общая точность модели составляет 0.64
- Также, как алгоритм Градиентный бустинг (Gradient Boosting), метод нейронных сетей демонстрируют высокие значения точности, полноты и F1-меры: Tinkoff Pay Wallet (0.94), Рассрочка на платежной форме (0.95) и Онлайн-касса (0.72) имеют высокие показатели.Также, категории СБП (0.76) и Реестры, акты и выгрузки (0.75) имеют хорошие результаты.
- Некоторые категории демонстрируют низкие значения точности, полноты и F1-меры: 3DS (0.16), SDK (0.38) и Маркетплейс (переводы) (0.12) имеют низкие показатели. Вне компетенции поддержки ИЭ (0.09) также имеет низкую точность и полноту.
- Модель показывает средние значения метрик, как macro avg, так и weighted avg, соответственно 0.54 и 0.63.

Исходя из этих результатов, можно сделать вывод, что модель нейронных сетей имеет средние показатели в предсказании различных категорий.


### Метод максимальной энтропии (Maximum Entropy)

In [ ]:
%%time

# Шаг 3: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 4: Обучение модели метода максимальной энтропии
max_ent = LogisticRegression()
max_ent.fit(X_train_vect, y_train)

# Шаг 5: Оценка модели метода максимальной энтропии
y_pred_max_ent = max_ent.predict(X_test_vect)
print("Отчет модели метода максимальной энтропии:")
print(classification_report(y_test, y_pred_max_ent))


Можно сделать следующие выводы:

1. Точность (precision): Некоторые классы имеют высокую точность (например, "BNPL (Долями)", "Origination", "Tinkoff ID", "Tinkoff Pay Wallet", "Безопасная сделка", "Выплаты (A2C)", "Онлайн-касса", "Реестры, акты и выгрузки"), превышающую 0.75. Средняя точность по всем классам составляет около 0.71, что указывает на хорошую общую точность модели.
2. Полнота (recall): Некоторые классы имеют высокую полноту (например, "Origination", "Tinkoff Pay Wallet", "Модули CMS и виджеты", "Онлайн-касса", "Реестры, акты и выгрузки"), превышающую 0.7. Средняя полнота по всем классам составляет около 0.69, что указывает на неплохую общую полноту модели.
3. F1-мера (f1-score): Средняя F1-мера по всем классам составляет около 0.68, что указывает на хорошую сбалансированную производительность модели.

Вывод: Метод максимальной энтропии (Maximum Entropy) является релевантным и стоит рядом с Методы опорных векторов (Support Vector Machines, SVM) и Логистическая регрессия (LogisticRegression)


### Методы гибридной классификации

In [ ]:
%%time

# Шаг 4: Создание пайплайна для преобразования и классификации
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', SVC(kernel='linear'))
])

# Шаг 5: Обучение и оценка модели с помощью пайплайна
pipeline.fit(X_train, y_train)
y_pred_cross = pipeline.predict(X_test)
print("Отчет модели:")
print(classification_report(y_test, y_pred_cross))


Вывод:
- Общая точность модели составляет 0.69, что означает, что около 69% предсказаний были классифицированы правильно.
- Модель показывает средние значения метрик, как macro avg, так и weighted avg, соответственно 0.56 и 0.68.
- Метод гибридной классификации является одним из лидеров при анализе десяти разный способов алгоритмизации обучения модели. Дале будем рассматривать его более детально с применением наилучших показателей гиперпараметров



## Улучшение модели

После анализа текущих показателей было выдвинуто несколько гипотез
- удаление некоторых тем из исходной датасета. Цель увеличить качечество датасета (избавить от шума) и повысить процент предсказания от моделей
- улучшить модель в качетсве предобработки данных. При анализе видно, что классы с худшими показателями, явлются те классы, что имеют минимальное кол-во представленных текстовых примеров. Таким образом, дабы сбалансировать весь датасет, было принято решенеие использовать алгоритм балансировки классов Применение методов балансировки классов позволяет улучшить производительность модели, сделав ее более справедливой при работе с несбалансированными наборами данных. Существуют два основных подхода к балансировке классов: oversampling (увеличение примеров минорного класса) и undersampling (сокращение примеров мажорного класса).

При первой итерации можо выделать следующие аолгоритмы, которые мы будем тестировать дальше:
- Методы опорных векторов (Support Vector Machines, SVM), где общее accuracy f1-score  = 0,69 (отработало за 52 мин.)
- Методы гибридной классификации, где общее accuracy f1-score  = 0,69 (отаработало за 27 мин.)
- Логистическая регрессия (LogisticRegression), где общее accuracy f1-score  = 0,67 (отработало за 2 мин.)
- Метод максимальной энтропии (Maximum Entropy), где общее accuracy f1-score  = 0,67 (отработало за 1 мин.)


Гипотезы будем проверять параллельно друг другу.
Во-первых, выделил отлеьно файл, где удалил неакутальные темы: "Маркетплейс (переводы)", "3DS", "SberPay", "E-inv", "Tinkoff Pay Wallet", "Нерезиденты". Актуальный файл - dat_min_500_menshe_topicov.
Во-вторых, проанализировав темы по количетсву, где бОльшая часть классов является минорными по отношению к классам, как Оплата картой (прямые мерчанты) около 15к и CБП около 8к, было принято решение использовать метод oversampling для балансировки классов.
Oversampling (увеличение примеров минорного класса) - это процесс увеличения примеров минорного класса путем генерации новых записей на основе имеющихся примеров.


### Балансировка классов

In [ ]:
%%time
# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer(lowercase=True)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Балансировка классов с помощью oversampling
from imblearn.over_sampling import RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_vect, y_train)

# Шаг 6: Обучение модели логистической регрессии на увеличенных данных
classifier = LogisticRegression()
classifier.fit(X_train_resampled, y_train_resampled)

# Шаг 7: Оценка модели
y_pred = classifier.predict(X_test_vect)
print("Отчет модели логистической регрессии:")
print(classification_report(y_test, y_pred))


In [ ]:
%%time
# Шаг 1: Чтение данных из переработанного xlsx файла
workbook = load_workbook(filename='dat_min_500_menshe_topicov.xlsx')
sheet = workbook.active
data = []
for row in sheet.iter_rows(values_only=True):
    data.append(row)

df = pd.DataFrame(data, columns=['text', 'product'])


In [ ]:
%%time
# Вывод шкалы измерения признака "Продукт"
print("Уникальные значения 'product':", df['product'].nunique())
print(df['product'].value_counts())


### Предобработка данных


In [ ]:
%%time
# Шаг 2: Предварительная обработка текста
stop_words = list(stopwords.words('russian'))

stop_words.extend([
 'С уважением',
 'Группа МПС',
 'МПС',
 'Отдел сопровождения',
 'ПЦ',
 'Контакты',
 'Вернемся с ответом в понедельник до по МСК',
 'Вернемся с ответом',
 'МСК',
 'до',
 'по',
 'Мы зарегистрировали ваше обращение и возьмем его в работу в ближайшее время',
 'Номер заявки',
 'Ответы на популярные вопросы можно найти в Тинькофф Помощи',
 'Команда поддержки Тинькофф Касса',
 'Январь',
 'Февраль',
 'Март',
 'Апрель',
 'Май',
 'Июнь',
 'Июль',
 'Август',
 'Сентябрь',
 'Октябрь',
 'Ноябрь',
 'Декабрь',
 'ФИО',
 'Используй кейс',
 'Сразу добавь в копию почту почты из столбца справа если указано',
 'Комментарий',
 'Обращение создано через формы на',
 'Просьба',
 'Менеджер по развитию партнеров',
 'Отдела продаж',
 'Департамента платежных систем',
 'Наталия Алексуткина',
 'Моб',
 'Доп',
 'Это электронное письмо строго конфиденциально включая все вложения',
 'Ссылка на чат с клиентом',
 'Отсутствует',
 'Власов Александр',
 'Технический специалист',
 'Мы в и',
 'Информация содержащаяся в данном письме и приложенных к нему материалах составляет коммерческую тайну и является конфиденциальной в связи с чем воспроизведение копирование распространение или использование информации любым иным образом может осуществляться исключительно с согласия уполномоченного представителя',
 'Если данное письмо было получено вами ошибочно просим незамедлительно удалить письмо',
 'пожалуйста',
 'нажимайте на ответить всем не видно ваших ответов',
 'Менеджер по развитию бизнеса',
 'Менеджер по развитию',
 'Команда сопровождения платежей и переводов',
 'спасибо',
 'Заранее',
 'посмотрите',
 'запрос',
 'Уточните',
 'Отдел сверок',
 'Ссылка на чат с клиентом',
 'Возобновление задачи',
 'Можем',
 'Мы',
 'Руководитель проектов',
 'Департамент платежных систем',
 'Ростелеком',
 'Вопрос в работе',
 'Вернемся с ответом до',
 'Прошу',
 'В чем может быть проблема',
 'Поддержка продуктов для бизнеса',
 'Бэк офис',
 'скриншот',
 'прикрепляю',
 'Сообщите',
 'номер тикета',
 'Специалист по работе с ключевыми клиентами',
 'необходимо',
 'подскажите',
 'умеем',
 'помогите',
 'поступило обращение',
 'поддержка',
 'Полиса',
 'Дата и время события',
 'Подробное описание',
 'Пользуйся кейсами со страницы продукта',
 'Подробности',
 'Проблема',
 'Отправлено из Почты',
 'Исходное сообщение',
 'поставьте нам оценку по ссылке',
 'уточнить',
 'Группа контроля эквайринга',
 'Данное сообщение отправлено из',
 'В случае отсутствия доп вопросов к',
 'не отвечать на него',
 'а так же удалить рассылку из дальнейшей переписки',
 'можете',
 'подсказать',
 'с вашей стороны',
 'Старший специалист',
 'вопрос',
 'Инженер группы поддержки платежей',
 'Отдел сопровождения платежей и переводов',
 'вы',
 'оценить мою работу здесь',
 'оставить чаевые здесь',
 'Менеджер по работе с партнерами',
 'прощение',
 'прощения',
 'пишу',
 'Сектор поддержки бизнеса',
 'Отдел претензий',
 'Управление',
 'Департамент',
 'Департамент клиентского обслуживания',
 'для звонков из России',
 'для звонков из-за границы',
 'Факс',
 'из-за',
 'из за',
 'чат',
 'Я ваш клиент',
 'Заранее благодарю',
 'Приносим извинения за доставленные неудобства',
 'извинения',
 'Диалог',
 'Персональный менеджер',
 'Направление корпоративного бизнеса',
 'Моб',
 'Бизнес аналитик',
 'аналитик',
 'добрый день',
 'здравствуйте',
 'добрый',
 'привет',
 'коллеги',
 'тел',
 'доб',
 'доброе утро',
 'доброе',
 'добрый вечер',
 'колеги'])



lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    start_time = timeit.default_timer()
    if text is not None:
        tokens = word_tokenize(text.lower())
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]
        tokens = [token for token in tokens if token not in stop_words]

    # Добавление ключевых слов
    keywords = [
 'qr',
 'sbp',
 'notifications',
 'notification',
 'cloudkassir',
 'rrn',
 'api',
 'iss_serno',
 'sdk',
 'cms',
 'tinkoff-pay',
 'tinkoffpay',
 'mirpay',
 'tinkoff pay',
 'returns amount',
 'msm',
 'acq',
 'acqapi',
 'compliance',
 'ovd',
 'tilda',
 'bitrix',
 'orangedata',
 'orange data',
 'ofd ferma',
 'ferma',
 'ecomkassa',
 'advantshop',
 'amiro',
 'amocrm',
 'cs-cart',
 'diafan',
 'drupal commerce',
 'drupal ubercart',
 'ecshop',
 'ecwid',
 'getcourse',
 'joomla hikashop',
 'hostcms',
 'image cms',
 'insales',
 'joomla',
 'joomshopping',
 'virtuemart',
 'magento',
 'maxystore',
 'modx shopkeeper',
 'mogutacms',
 'netcat',
 'opencart',
 'oscommerce',
 'phpshop.cms',
 'prestashop',
 'quickbuy',
 'simpla',
 'taplink',
 'umi.cms',
 'vamshop',
 'vigbo',
 'webasyst',
 'wfolio',
 'wordpress ecommerce',
 'wordpress shop',
 'wordpress',
 'woocommerce',
 'seller',
 'chargeback',
 'mapi',
 'arn',
 'ssl',
 'mcc',
 'bnpl',
 'outward',
 'outwards',
 'atol',
 'webhook',
 'wl',
 'white list',
 'whitelist',
 'dolyame']

    tokens += keywords

    return ' '.join(tokens)

df['Processed_Text'] = df['text'].apply(preprocess_text)


### Применение машионного обучения с гепперпараметрами


In [ ]:
%%time
# Шаг 3: Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(df['Processed_Text'], df['product'], test_size=0.2, random_state=42)


В текущей итерации будем рассматривать следующие алгоритмы:
- Методы опорных векторов (Support Vector Machines, SVM)
- Методы гибридной классификации
- Логистическая регрессия (LogisticRegression)
- Метод максимальной энтропии (Maximum Entropy)
В рамках данной итерации сразу применю перекрестную проверку (cross_val_score)для достоверности измерения метрик моделей.

При обучении моделей машинного обучения часто требуется оценить, насколько хорошо модель обобщает данные и способна предсказывать правильные результаты для новых данных. Однако при разделении данных на тренировочный и тестовый наборы возникает проблема влияния конкретной случайной выборки на результаты оценки модели. Если случайная выборка не репрезентативна или содержит смещенность (например, в одном наборе данных преобладает один класс), то точность модели на отложенном тестовом наборе может недостоверно отражать ее реальную производительность.

Чтобы избежать этой проблемы, вместо единственного разделения данных на тренировочный и тестовый наборы, используется перекрестная проверка. Перекрестная проверка позволяет повторить обучение и оценку модели на различных разбиениях данных, чтобы получить более надежные оценки производительности модели.

После выполнения перекрестной проверки выводится точность на каждой итерации и средняя точность на перекрестной проверке. Средняя точность позволяет оценить обобщающую способность модели на основе всех разбиений данных и будет более надежной мерой ее производительности на новых данных. Поэтому перекрестная проверка помогает оценить и выбрать модель с наиболее надежными результатами и определить лучшие гиперпараметры для этой модели.


### Методы опорных векторов (Support Vector Machines, SVM) с использованием GridSearchCV


In [ ]:
%%time

# Шаг 4: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 5: Обучение модели Методов опорных векторов (SVM) с использованием GridSearchCV
svm = SVC()
parameters = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
grid_search = GridSearchCV(svm, parameters, cv=5)
grid_search.fit(X_train_vect, y_train)

# Получение лучших параметров и лучшей оценки модели
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Лучшие параметры:", best_params)
print("Лучшая оценка модели:", best_score)

# Шаг 6: Оценка модели с лучшими параметрами
best_svm = grid_search.best_estimator_
y_pred_svm_giperparam = best_svm.predict(X_test_vect)
print("Отчет модели опорных векторов с лучшими параметрами:")
print(classification_report(y_test, y_pred_svm_giperparam))


В данном коде, создаю объект GridSearchCV с указанными значениями гиперпараметров для перебора (C и gamma). Затем, используя метод fit(), обучаю модель SVM с каждой комбинацией гиперпараметров. GridSearchCV автоматически выполнит перекрестную проверку (cross-validation) для каждой комбинации и выберет модель с наилучшей оценкой.

После завершения GridSearchCV, мы получаем лучшие параметры (best_params) и лучшую оценку модели (best_score). Затем, используя best_params, создаю модель SVM с лучшими параметрами (best_svm).

При использовании GridSearchCV было выявлено, что код выполняется большое время и тарит производительность ноутбук, в связи с этим было принято решение использовать RandomizedSearchCV вместо GridSearchCV для поиска лучших гиперпараметров. RandomizedSearchCV осуществляет случайный поиск по заданному пространству гиперпараметров. Данный метод считаю боле эффективным в нашем случае, где пространство поиска очень большое.


### Методы гибридной классификации с использованием RandomizedSearchCV и перекрестной проверки


In [ ]:
%%time

# Шаг 4: Создание пайплайна для преобразования и классификации
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', SVC())
])

# Шаг 5: Определение гиперпараметров для поиска
parameters = {'vectorizer__lowercase': [True, False], 'classifier__C': [0.1, 1, 10], 'classifier__gamma': [0.1, 1, 10]}

# Шаг 6: Применение RandomizedSearchCV для поиска лучших гиперпараметров
random_search = RandomizedSearchCV(pipeline, parameters, cv=5)
random_search.fit(X_train, y_train)

# Получение лучших параметров и лучшей оценки модели
best_params = random_search.best_params_
best_score = random_search.best_score_
print("Лучшие параметры:", best_params)
print("Лучшая оценка модели:", best_score)

# Шаг 7: Оценка модели с лучшими параметрами
best_pipeline = random_search.best_estimator_
y_pred_giperparam = best_pipeline.predict(X_test)
print("Отчет модели с лучшими параметрами:")
print(classification_report(y_test, y_pred_giperparam))

# Шаг 8: Применение перекрестной проверки для оценки модели
cv_scores = cross_val_score(best_pipeline, X_train, y_train, cv=5, scoring='accuracy')
print("Точность на каждой итерации перекрестной проверки:", cv_scores)
print("Средняя точность на перекрестной проверке:", cv_scores.mean())


### Логистическая регрессия (LogisticRegression) с использованием RandomizedSearchCV и перекрестной проверки


In [ ]:
%%time


# Шаг 4: Создание пайплайна для преобразования и классификации
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

# Шаг 5: Определение гиперпараметров для поиска
parameters = {
    'vectorizer__lowercase': [True, False],
    'vectorizer__max_features': [1000, 5000, 10000],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l1', 'l2']
}

# Шаг 6: Применение RandomizedSearchCV для поиска лучших гиперпараметров
random_search = RandomizedSearchCV(pipeline, parameters, cv=5)
random_search.fit(X_train, y_train)

# Шаг 7: Оценка модели с лучшими параметрами
best_params = random_search.best_params_
print("Лучшие параметры:", best_params)

# Использование модели с лучшими параметрами для предсказания
best_pipeline_lr = random_search.best_estimator_
y_pred_lr_giperparam = best_pipeline_lr.predict(X_test)
print("Отчет модели с лучшими параметрами:")
print(classification_report(y_test, y_pred_lr_giperparam))

# Шаг 8: Применение перекрестной проверки для оценки модели
cv_scores = cross_val_score(best_pipeline_lr, X_train, y_train, cv=5, scoring='accuracy')
print("Точность на каждой итерации перекрестной проверки:", cv_scores)
print("Средняя точность на перекрестной проверке:", cv_scores.mean())


Анализируя предоставленную информацию, можно сделать следующие выводы:
1. Лучшие параметры для модели:
    - vectorizer__ngram_range: (1, 1)
    - vectorizer__max_features: 5000
    - vectorizer__lowercase: True
    - classifier__penalty: 'l2'
    - classifier__C: 10
Эти параметры указывают на использование униграмм, ограничение до 5000 наиболее часто встречающихся признаков, преобразование текста в нижний регистр, использование регуляризации l2 и значения параметра C равное 10.
2. Отчет модели с лучшими параметрами предоставляет информацию о точности, полноте, F1-мере и поддержке для каждой категории классов: в таблице отчета видны относительно высокие значения точности, полноты и F1-меры для некоторых категорий, таких как "Рассрочка на платежной форме" и "СБП". Однако, для некоторых категорий, таких как "Личный кабинет" и "Маркетплейс", значения этих метрик ниже.
3. В целом, средние значения точности, полноты и F1-меры для всех категорий составляют около 0.72, что говорит о том, что модель имеет стабильную производительность для всех категорий классов. Общая точность модели составляет около 0.72, что означает, что примерно 72% всех предсказанных меток были правильными.
4. Время выполнения модели составило около 20 минут и 32 секунды, что является более быстрым временным затратом по сравнению с предыдущим примером.

Таким образом, модель с лучшими параметрами показывает приемлемую производительность, с некоторым дисбалансом в точности и полноте для некоторых категорий. Общая производительность модели сопоставима с прошлым примером, однако время выполнения значительно сократилось.


### Метод максимальной энтропии (Maximum Entropy) с использованием GridSearchCV и перекрестной проверки


In [ ]:
%%time


# Шаг 3: Создание векторизатора для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 4: Определение модели и параметров для поиска
model = LogisticRegression()
parameters = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}

# Шаг 5: Применение GridSearchCV для поиска лучших гиперпараметров
grid_search = GridSearchCV(model, parameters, cv=5)
grid_search.fit(X_train_vect, y_train)

# Шаг 6: Оценка модели с лучшими параметрами
best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)

# Шаг 7: Использование модели с лучшими параметрами для предсказания
best_model_en = grid_search.best_estimator_
y_pred_max_ent_giperparam = best_model_en.predict(X_test_vect)

# Шаг 8: Оценка модели
print("Отчет модели с лучшими параметрами:")
print(classification_report(y_test, y_pred_max_ent_giperparam))

# Шаг 9: Применение перекрестной проверки для оценки модели
cv_scores = cross_val_score(best_model_en, X_train_vect, y_train, cv=5, scoring='accuracy')
print("Точность на каждой итерации перекрестной проверки:", cv_scores)
print("Средняя точность на перекрестной проверке:", cv_scores.mean())


Анализируя информацию можно сделать следующие выводы:
1. Лучшие параметры для модели: {'C': 10, 'penalty': 'l2'}. Это означает, что модель достигла наилучшей производительности при использовании значений параметров C=10 и penalty=l2.
2. В таблице отчета видны относительно высокие значения точности, полноты и F1-меры для некоторых категорий, таких как "Рассрочка на платежной форме" и "СБП". Однако для некоторых категорий, таких как "Личный кабинет" и "Маркетплейс", значения этих метрик ниже.
3. В целом, средние значения точности, полноты и F1-меры для всех категорий составляют около 0.72, что говорит о том, что модель имеет стабильную производительность для всех категорий классов.
4. Общая точность модели составляет около 0.73, что означает, что примерно 73% всех предсказанных меток были правильными.
5. Время выполнения модели составило около 14 минут и 24 секунды, что является наименьшим временным затратом среди всех предоставленных примеров.

Сравнивая эти результаты с предыдущими примерами, можно заметить, что модель с лучшими параметрами C=10 и penalty=l2 имеет примерно схожую производительность со всеми рассмотренным примерами.
Время выполнения данной модели было самым низким среди предоставленных примеров, что свидетельствует о ее эффективности.


# Визуализация результатов моделирования


Визуализация результатов моделирования является важной частью анализа данных и может быть полезной для понимания и интерпретации производительности модели. Вот некоторые преимущества и цели визуализации результатов моделирования:


### Матрица ошибок (Confusion Matrix)
Визуализация матрицы ошибок: Матрица ошибок позволяет визуализировать сравнение между фактическими значениями и предсказанными значениями модели. Она отображает количество верных и неверных классификаций для каждого класса, что позволяет оценить точность модели, обнаружить классы с высокими или низкими показателями точности, а также выявить наличие систематических ошибок.
Представление важных признаков: Визуализация важных признаков в модели может помочь в понимании, какие факторы имеют наибольшее влияние на предсказания модели. Это может быть полезно для принятия более информированных решений и понимания важности различных переменных.


In [ ]:
# Шаг 9: Построение матрицы ошибок
confusion_matrix = pd.crosstab(y_test, y_pred_max_ent_giperparam, rownames=['Actual'], colnames=['Predicted'])
plt.figure(figsize=(12, 8))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.title("Матрица ошибок")
plt.show()



Диаграмма box-plot и violin-plot
Данный тип диаграмм полез нам для сравнения распределения прогнозов для разных классов и могут дать представление о том, насколько хорошо модель разделяет классы.


In [ ]:
%%time

# Шаг 9: Визуализация диаграмм box-plot и violin-plot
plt.figure(figsize=(8, 6))
plt.boxplot([cv_scores], showmeans=True)
plt.xticks([1], ['Cross Validation'])
plt.ylabel('Accuracy')
plt.title('Ящик для оценок перекрестной проверки')
plt.show()

plt.figure(figsize=(8, 6))
plt.violinplot(cv_scores, showmeans=True)
plt.xticks([1], ['Cross Validation'])
plt.ylabel('Accuracy')
plt.title('График результатов перекрестной проверки для скрипки')
plt.show()


# Проверка на чистых данных


In [ ]:
%%time
# Шаг 6: Предсказание категории для нового сообщения
new_message = "Просим предоставить документы подтверждающие успешность транзакции"
new_message = preprocess_text(new_message)
new_message_vec = vectorizer.transform([new_message])
predicted_category = best_model_en.predict(new_message_vec)[0]
print('Predicted Category:', predicted_category)


In [ ]:
%%time

# Функция для предобработки текста (проверил даже с предобработкой текста получаются одинаковые результаты)
def preprocess_text(text):
    # Приведение к нижнему регистру
    #text = text.lower()

    # Токенизация
    #tokens = word_tokenize(text)

    # Лемматизация текста
    #lemmatizer = WordNetLemmatizer()
    #lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Объединение лемматизированных токенов обратно в строку
    #processed_text = ' '.join(lemmatized_tokens)
    return text

# Загрузка данных из эксель файла
df = pd.read_excel('prediction_data_min_500.xlsx')
texts = df.iloc[:, 0].tolist()

# Создание списка для хранения предсказанных категорий
predicted_categories = []


for text in texts:
    processed_text = preprocess_text(text)
    text_vector = vectorizer.transform([processed_text])
    text_vector = text_vector.astype('float32')  # Преобразование в тип float32
    predicted_category = best_model_en.predict(text_vector)[0]
    predicted_categories.append(predicted_category)

# Добавление предсказанных категорий в третью колонку датафрейма
df['Predicted Category'] = predicted_categories

# Сохранение данных с предсказанными категориями обратно в эксель файл
df.to_excel('prediction_data_min_500.xlsx', index=False)
print('Данные с предсказанием сохранены в файле "prediction_data_min_500.xlsx"')


In [ ]:
%%time

# Загрузка данных из эксель файла
df = pd.read_excel('prediction_data_min_500.xlsx')

# Создание счетчика для каждого продукта
product_counts = {}
product_matching_counts = {}
product_total_counts = {}

# Инициализация счетчиков
for product in df['product'].unique():
    product_counts[product] = 0
    product_matching_counts[product] = 0
    product_total_counts[product] = 0

# Подсчет количества каждого продукта и сравнение значений
for index, row in df.iterrows():
    product = row['product']
    product_counts[product] += 1
    product_total_counts[product] += 1

    if row['product'] == row['Predicted Category']:  # Сравнение текста с продуктом
        product_matching_counts[product] += 1

total_matching_count = sum(product_matching_counts.values())
total_non_matching_count = sum(product_total_counts.values()) - total_matching_count

total_count = len(df)  # Общее количество значений

# Функция для сортировки по проценту совпадений
def sort_by_matching_percentage(item):
    product = item[0]
    product_matching_count = item[1]
    return (product_matching_count / product_total_counts[product]) * 100, product

# Вывод результатов по каждому продукту, отсортированных по проценту совпадений
for product, product_matching_count in sorted(product_matching_counts.items(), key=sort_by_matching_percentage, reverse=True):
    product_non_matching_count = product_counts[product] - product_matching_count
    product_matching_percentage = (product_matching_count / product_total_counts[product]) * 100
    print(f"Для продукта {product}:")
    print(f"- Количество совпадающих значений: {product_matching_count}")
    print(f"- Количество несовпадающих значений: {product_non_matching_count}")
    print(f"- Процент совпадающих значений: {product_matching_percentage:.2f}%")
    print()

# Вывод общих результатов
print("Общие результаты:")
print(f"Количество совпадающих значений: {total_matching_count}")
print(f"Количество несовпадающих значений: {total_non_matching_count}")
print(f"Процент совпадающих значений: {(total_matching_count / total_count) * 100:.2f}%")

In [ ]:
#Чтение данных из xlsx файла
workbook = load_workbook(filename='product_matching_results_dat_min_500_prod.xlsx')
sheet = workbook.active
data = []
for row in sheet.iter_rows(values_only=True):
    data.append(row)

df = pd.DataFrame(data, columns=['Продукт', 'Процент совпадающих значений', 'Количество значений'])


In [ ]:
# выводим данные
df